In [1]:
!unzip "BirdNET-Lite.zip"

Archive:  BirdNET-Lite.zip
  inflating: analyze.py              
  inflating: __MACOSX/._analyze.py   
   creating: Audio_example/
  inflating: Audio_example/.DS_Store  
  inflating: __MACOSX/Audio_example/._.DS_Store  
  inflating: Audio_example/4.wav     
  inflating: __MACOSX/Audio_example/._4.wav  
  inflating: Audio_example/1.wav     
  inflating: __MACOSX/Audio_example/._1.wav  
  inflating: Audio_example/3.wav     
  inflating: __MACOSX/Audio_example/._3.wav  
  inflating: Audio_example/2.wav     
  inflating: __MACOSX/Audio_example/._2.wav  
   creating: Audio_result/
  inflating: __MACOSX/._Audio_result  
  inflating: Audio_result/.DS_Store  
  inflating: __MACOSX/Audio_result/._.DS_Store  
  inflating: LICENSE                 
  inflating: __MACOSX/._LICENSE      
   creating: model/
  inflating: __MACOSX/._model        
  inflating: model/labels.txt        
  inflating: __MACOSX/model/._labels.txt  
  inflating: model/BirdNET_6K_GLOBAL_MODEL.tflite  
  inflating: __MACOSX/mo

In [2]:
! pip uninstall -y jax jaxlib tensorflow tflite-runtime numpy

Found existing installation: jax 0.5.2
Uninstalling jax-0.5.2:
  Successfully uninstalled jax-0.5.2
Found existing installation: jaxlib 0.5.1
Uninstalling jaxlib-0.5.1:
  Successfully uninstalled jaxlib-0.5.1
Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2


In [3]:
! pip cache purge
! pip install soundfile resampy numpy==1.24.4 tensorflow==2.15.0

Files removed: 0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Suc

# CPU TEST

In [1]:
import os
import time
from datetime import datetime

AUDIO_FOLDER = "Audio_example"
OUTPUT_FOLDER = "Audio_result"
lat = 13.75
lon = 100.5
week = 18

# Start timer monitoring
start = time.time()

# Process one by one (sequentially)
for file in os.listdir(AUDIO_FOLDER):
    if file.endswith(".wav"):
        input_path = os.path.join(AUDIO_FOLDER, file)
        output_path = os.path.join(OUTPUT_FOLDER, file.replace(".wav", ".csv"))
        cmd = f"python analyze.py --i {input_path} --o {output_path} --lat {lat} --lon {lon} --week {week}"

        start_time = datetime.now()
        print(f"[{start_time.strftime('%H:%M:%S')}] START: {cmd}")
        os.system(cmd)
        end_time = datetime.now()

        duration = (end_time - start_time).total_seconds()
        print(f"[{end_time.strftime('%H:%M:%S')}] END: {file} processed in {duration:.2f} seconds\n")

end = time.time()
print(f"Total Time (Single-threaded): {end - start:.2f} seconds")

[20:46:02] START: python analyze.py --i Audio_example/4.wav --o Audio_result/4.csv --lat 13.75 --lon 100.5 --week 18
[20:46:16] END: 4.wav processed in 13.34 seconds

[20:46:16] START: python analyze.py --i Audio_example/3.wav --o Audio_result/3.csv --lat 13.75 --lon 100.5 --week 18
[20:46:22] END: 3.wav processed in 6.52 seconds

[20:46:22] START: python analyze.py --i Audio_example/2.wav --o Audio_result/2.csv --lat 13.75 --lon 100.5 --week 18
[20:46:31] END: 2.wav processed in 9.19 seconds

[20:46:31] START: python analyze.py --i Audio_example/1.wav --o Audio_result/1.csv --lat 13.75 --lon 100.5 --week 18
[20:46:40] END: 1.wav processed in 8.63 seconds

Total Time (Single-threaded): 37.69 seconds


In [2]:
# Multithreaded execution: Run up to 2 audio files in parallel using a thread pool
import os
import time
import subprocess
import threading
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

AUDIO_FOLDER = "Audio_example"
OUTPUT_FOLDER = "Audio_result"
lat = 13.75
lon = 100.5
week = 18

def run_analysis(file):
    try:
        input_path = os.path.join(AUDIO_FOLDER, file)
        output_path = os.path.join(OUTPUT_FOLDER, file.replace(".wav", ".csv"))

        if not os.path.exists(input_path):
            raise FileNotFoundError(f"{input_path} not found")
        if not file.endswith(".wav"):
            raise ValueError(f"{file} is not a WAV file")

        command = [
            "python", "analyze.py",
            "--i", input_path,
            "--o", output_path,
            "--lat", str(lat),
            "--lon", str(lon),
            "--week", str(week)
        ]

        start_time = datetime.now()
        print(f"[{start_time.strftime('%H:%M:%S')}] START: {' '.join(command)}")

        result = subprocess.run(command, capture_output=True, text=True)

        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds()

        if result.returncode != 0:
            print(f"[{end_time.strftime('%H:%M:%S')}] Error processing {file} (Duration: {duration:.2f}s):\n{result.stderr}")
        else:
            print(f"[{end_time.strftime('%H:%M:%S')}] Processed {file} successfully in {duration:.2f} seconds")

    except Exception as e:
        print(f"Failed to process {file}: {e}")

# Run multithreaded analysis
wav_files = [f for f in os.listdir(AUDIO_FOLDER) if f.endswith(".wav")]
start = time.time()
with ThreadPoolExecutor(max_workers=2) as executor:
    executor.map(run_analysis, wav_files)
end = time.time()

print(f"Total Time (Multithreaded): {end - start:.2f} seconds")


[20:46:47] START: python analyze.py --i Audio_example/4.wav --o Audio_result/4.csv --lat 13.75 --lon 100.5 --week 18
[20:46:47] START: python analyze.py --i Audio_example/3.wav --o Audio_result/3.csv --lat 13.75 --lon 100.5 --week 18
[20:46:59] Processed 3.wav successfully in 11.99 seconds
[20:46:59] START: python analyze.py --i Audio_example/2.wav --o Audio_result/2.csv --lat 13.75 --lon 100.5 --week 18
[20:47:03] Processed 4.wav successfully in 15.97 seconds
[20:47:03] START: python analyze.py --i Audio_example/1.wav --o Audio_result/1.csv --lat 13.75 --lon 100.5 --week 18
[20:47:13] Processed 2.wav successfully in 14.01 seconds
[20:47:16] Processed 1.wav successfully in 12.93 seconds
Total Time (Multithreaded): 28.89 seconds


In [4]:
# Multithreaded execution: Run up to 4 audio files in parallel using a thread pool
import os
import time
import subprocess
import threading
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

AUDIO_FOLDER = "Audio_example"
OUTPUT_FOLDER = "Audio_result"
lat = 13.75
lon = 100.5
week = 18

def run_analysis(file):
    try:
        input_path = os.path.join(AUDIO_FOLDER, file)
        output_path = os.path.join(OUTPUT_FOLDER, file.replace(".wav", ".csv"))

        if not os.path.exists(input_path):
            raise FileNotFoundError(f"{input_path} not found")
        if not file.endswith(".wav"):
            raise ValueError(f"{file} is not a WAV file")

        command = [
            "python", "analyze.py",
            "--i", input_path,
            "--o", output_path,
            "--lat", str(lat),
            "--lon", str(lon),
            "--week", str(week)
        ]

        start_time = datetime.now()
        print(f"[{start_time.strftime('%H:%M:%S')}] START: {' '.join(command)}")

        result = subprocess.run(command, capture_output=True, text=True)

        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds()

        if result.returncode != 0:
            print(f"[{end_time.strftime('%H:%M:%S')}] Error processing {file} (Duration: {duration:.2f}s):\n{result.stderr}")
        else:
            print(f"[{end_time.strftime('%H:%M:%S')}] Processed {file} successfully in {duration:.2f} seconds")

    except Exception as e:
        print(f"Failed to process {file}: {e}")

# Run multithreaded analysis
wav_files = [f for f in os.listdir(AUDIO_FOLDER) if f.endswith(".wav")]
start = time.time()
with ThreadPoolExecutor(max_workers=4) as executor:
    executor.map(run_analysis, wav_files)
end = time.time()

print(f"Total Time (Multithreaded): {end - start:.2f} seconds")


[20:48:03] START: python analyze.py --i Audio_example/4.wav --o Audio_result/4.csv --lat 13.75 --lon 100.5 --week 18
[20:48:03] START: python analyze.py --i Audio_example/3.wav --o Audio_result/3.csv --lat 13.75 --lon 100.5 --week 18
[20:48:03] START: python analyze.py --i Audio_example/2.wav --o Audio_result/2.csv --lat 13.75 --lon 100.5 --week 18
[20:48:03] START: python analyze.py --i Audio_example/1.wav --o Audio_result/1.csv --lat 13.75 --lon 100.5 --week 18
[20:48:31] Processed 3.wav successfully in 27.71 seconds
[20:48:32] Processed 2.wav successfully in 29.06 seconds
[20:48:33] Processed 1.wav successfully in 29.51 seconds
[20:48:35] Processed 4.wav successfully in 31.66 seconds
Total Time (Multithreaded): 31.66 seconds
